In [1]:
filename = 'Devoluciones Complemento (Carga Yosaf) 2022 Marzo.xlsx'
year = 2022

# Libraries

In [2]:
import pandas as pd
import pyodbc
import os
from datetime import datetime

pd.io.formats.excel.ExcelFormatter.header_style = None
pd.options.display.float_format = '{:,.2f}'.format

# Parameters

In [3]:
path = r'\\NASPRO.infovisiontv.com\DGI\DGI Banco Central\00 CIERRES SEMANALES\{0}\Devoluciones'
path = path.format(year)

# Import Data

'Grupo de artículos'

In [4]:
%%time
df = pd.read_excel(path + '\\' + filename, sheet_name='YOSAF', dtype={'Grupo de artículos':'str', 'Tp.cambio p.contab.':'str', 'Lote':'str'})

Wall time: 5.55 s


df[['Grupo de artículos', 'Tp.cambio p.contab.', 'Lote']].dtypes

In [5]:
df.shape

(2999, 47)

# Data Cleaning

df = df[:df.shape[0]-6]

In [6]:
df.isnull().sum()[df.isnull().sum() > 0]

Doc.fact.              2999
Documento de ventas    2999
Referencia cliente     2999
Prc.Unitario           2999
Imp.Base               2999
Imp.Descuento          2999
Imp.Impuesto           2999
Imp.IEPS               2999
Imp.Total              2999
Centro de beneficio    2999
Descripción CeBe       2999
Grupo de artículos     2999
Tp.cambio p.contab.    2999
Lote                   2999
dtype: int64

In [7]:
new_cols = [
     'Factura',
     'Posicion',
     'DocumentoVentas',
     'PosicionDV',
     'NumeroPedCliente',
     'Material',
     'CodigoBarras',
     'Denominacion',
     'Solicitante',
     'Nombre',
     'RFC',
     'CNAC',
     'GrupoCuentas',
     'GrupoTesoreria',
     'DestinatarioMcia',
     'NombreDestinatario',
     'CantidadFacturada',
     'UnidadMedida',
     'CtdFactUMB',
     'UnidadMedidaBase',
     'PrecioUnitario',
     'ImpuestoBase',
     'ImpuestoDescuento',
     'ValorNeto',
     'ImporteImpuesto',
     'ImpuestoIEPS',
     'ImpuestoTotal',
     'CentroBeneficio',
     'DescripcionCebe',
     'TipoUnidades',
     'FechaFactura',
     'GrupoArticulos',
     'DenominacionArticulos',
     'ClaseFactura',
     'DenominacionTipoDocto',
     'CambioContable',
     'Sociedad',
     'Nombreempresa',
     'OrganizacionVentas',
     'Centro',
     'Almacen',
     'Lote',
     'FechaCaducidad',
     'MotivoPedido',
     'DescMotivoPedido',
     'ValorNetoAbs',
     'Moneda'
]

In [8]:
df.columns = new_cols

In [9]:
cols_nulls = [ 
    'NumeroPedCliente', 
    'RFC', 
    'CNAC', 
    'Almacen', 
    'Lote', 
    'MotivoPedido', 
    'DescMotivoPedido', 
    'GrupoArticulos',
    'CentroBeneficio',
    'DescripcionCebe'
]

for col in cols_nulls: 
    df[col].fillna('     ', inplace=True)

In [10]:
cols_int = [
    'Factura',
    'Posicion',
    'DocumentoVentas',
    'PosicionDV',
    'Material',
    'Solicitante',
    'CtdFactUMB',
    'ImpuestoDescuento',
    'ImpuestoIEPS',
    'PrecioUnitario',
    'ImpuestoBase',
    'ImporteImpuesto',
    'ImpuestoTotal',
    'CambioContable'
]

for col in cols_int: 
    df[col].fillna(0, inplace=True)
    df[col] = df[col].map(int)

In [11]:
df.isnull().sum()[df.isnull().sum() > 0]

Series([], dtype: int64)

In [12]:
df[['FechaCaducidad', 'FechaCaducidad']].dtypes

FechaCaducidad    datetime64[ns]
FechaCaducidad    datetime64[ns]
dtype: object

# Analysis

In [13]:
print('FechaFactura desde:', df['FechaFactura'].min())
print('FechaFactura hasta:', df['FechaFactura'].max())

FechaFactura desde: 2022-03-31 00:00:00
FechaFactura hasta: 2022-03-31 00:00:00


In [14]:
print('FechaCaducidad desde:', df['FechaCaducidad'].min())
print('FechaCaducidad hasta:', df['FechaCaducidad'].max())

FechaCaducidad desde: 2021-12-31 00:00:00
FechaCaducidad hasta: 2021-12-31 00:00:00


# Export Data

In [15]:
df.to_excel(path + '\\Cargar ' + filename.split('.')[0] + '.xlsx',
            index=False,
            sheet_name='carga')

In [16]:
df.shape

(2999, 47)

# Validations

In [17]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Trusted_Connection=yes;')

In [18]:
qtime = '''SELECT DISTINCT 
                TmpFecha,
                TmpSemanaAnioGenomma AS SemanaGL,
                TmpAnioSemanaGenomma AS AnioGL
            FROM Gnm_DWH.dbo.Dim_Tiempo'''

In [19]:
dtime = pd.read_sql(qtime, conn)

In [20]:
df = df.merge(dtime, left_on='FechaFactura', right_on='TmpFecha', how='left')

In [21]:
df.pivot_table(
    index='SemanaGL', 
    values=['ValorNeto'],
    aggfunc='sum'
)

,ValorNeto
SemanaGL,
14,"-5,790,367.55"


In [22]:
df.pivot_table(
    index='SemanaGL', 
    values='Factura',
    aggfunc='count',
    margins=True
)

,Factura
SemanaGL,
14,2999
All,2999


In [23]:
df.pivot_table(
    index='FechaFactura', 
    values='Factura',
    aggfunc='count',
    margins=True
)

,Factura
FechaFactura,
2022-03-31 00:00:00,2999
All,2999


In [24]:
df.pivot_table(
    index='MotivoPedido', 
    values='ValorNeto', 
    aggfunc='sum'
)

,ValorNeto
MotivoPedido,
M22,"-5,790,367.55"


In [25]:
df.isnull().sum()[df.isnull().sum() > 0]

Series([], dtype: int64)